![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Working and Long-Term Memory

**⏱️ Estimated Time:** 45-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why memory is essential for context engineering
2. **Implement** working memory for conversation continuity
3. **Use** long-term memory for persistent user knowledge
4. **Integrate** memory with your Section 2 RAG system
5. **Build** a complete memory-enhanced course advisor

---

## 🔗 Recap

### **Section 1: The Four Context Types**

Recall the four context types from Section 1:

1. **System Context** (Static) - Role, instructions, guidelines
2. **User Context** (Dynamic, User-Specific) - Profile, preferences, goals
3. **Conversation Context** (Dynamic, Session-Specific) - **← Memory enables this!**
4. **Retrieved Context** (Dynamic, Query-Specific) - RAG results

### **Section 2: Stateless RAG**

Your Section 2 RAG system was **stateless**:

```python
async def rag_query(query, student_profile):
    # 1. Search courses (Retrieved Context)
    courses = await course_manager.search_courses(query)

    # 2. Assemble context (System + User + Retrieved)
    context = assemble_context(system_prompt, student_profile, courses)

    # 3. Generate response
    response = llm.invoke(context)

    # ❌ No conversation history stored
    # ❌ Each query is independent
    # ❌ Can't reference previous messages
```

**The Problem:** Every query starts from scratch. No conversation continuity.

---

## 🚨 Why Agents Need Memory: The Grounding Problem

Before diving into implementation, let's understand the fundamental problem that memory solves.

**Grounding** means understanding what users are referring to. Natural conversation is full of references:

### **Without Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers supervised learning..."

User: "What are its prerequisites?"
Agent: ❌ "What does 'it' refer to? Please specify which course."

User: "The course we just discussed!"
Agent: ❌ "I don't have access to previous messages. Which course?"
```

**This is a terrible user experience.**

### Types of References That Need Grounding

**Pronouns:**
- "it", "that course", "those", "this one"
- "he", "she", "they" (referring to people)

**Descriptions:**
- "the easy one", "the online course"
- "my advisor", "that professor"

**Implicit context:**
- "Can I take it?" → Take what?
- "When does it start?" → What starts?

**Temporal references:**
- "you mentioned", "earlier", "last time"

### **With Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers..."
[Stores: User asked about CS401]

User: "What are its prerequisites?"
Agent: [Checks memory: "its" = CS401]
Agent: ✅ "CS401 requires CS201 and MATH301"

User: "Can I take it?"
Agent: [Checks memory: "it" = CS401, checks student transcript]
Agent: ✅ "You've completed CS201 but still need MATH301"
```

**Now the conversation flows naturally!**

---

## 🧠 Two Types of Memory

### **1. Working Memory (Session-Scoped)**

 - **What:** Conversation messages from the current session
 - **Purpose:** Reference resolution, conversation continuity
 - **Lifetime:** Persists for the session
 - **Storage:** Conversation remains accessible when you return to the same session

**Example:**
```
Session: session_123
Messages:
  1. User: "Tell me about CS401"
  2. Agent: "CS401 is Machine Learning..."
  3. User: "What are its prerequisites?"
  4. Agent: "CS401 requires CS201 and MATH301"
```

**Key Point:** Just like ChatGPT or Claude, when you return to a conversation, the working memory is still there. The conversation doesn't disappear!

### **2. Long-term Memory (Cross-Session)**

 - **What:** Persistent knowledge (user preferences, domain facts, business rules)
 - **Purpose:** Personalization AND consistent application behavior across sessions
 - **Lifetime:** Permanent (until explicitly deleted)
 - **Scope:** Can be user-specific OR application-wide

**Examples:**

**User-Scoped (Personalization):**
```
User: student_sarah
  - "Prefers online courses over in-person"
  - "Major: Computer Science, focus on AI/ML"
  - "Goal: Graduate Spring 2026"
  - "Completed: CS101, CS201, MATH301"
```

**Application-Scoped (Domain Knowledge):**
```
Domain: course_requirements
  - "CS401 requires CS201 as prerequisite"
  - "Maximum course load is 18 credits per semester"
  - "Registration opens 2 weeks before semester start"
  - "Lab courses require campus attendance"
```

### **Comparison: Working vs. Long-term Memory**

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped OR Application-scoped** |
| Current conversation | Important facts, rules, knowledge |
| Persists for session | Persists across sessions |
| Full message history | Extracted knowledge (user + domain) |
| Loaded/saved each turn | Searched when needed |
| **Challenge:** Context window limits | **Challenge:** Storage growth |

---

## 📦 Setup and Environment

Let's set up our environment with the necessary dependencies and connections. We'll build on Section 2's RAG foundation and add memory capabilities.

### ⚠️ Prerequisites

**Before running this notebook, make sure you have:**

1. **Docker Desktop running** - Required for Redis and Agent Memory Server

2. **Environment variables** - Create a `.env` file in the project root directory:
   ```bash
   # Copy the example file (if it exists)
   cd ../..
   # Or create .env manually with:
   # OPENAI_API_KEY=your_actual_openai_api_key_here
   # REDIS_URL=redis://localhost:6379
   # AGENT_MEMORY_URL=http://localhost:8088
   ```

3. **Start services** - Make sure Redis and Agent Memory Server are running:
   ```bash
   # Start Redis and Agent Memory Server using docker-compose
   cd ../..
   docker-compose up -d
   ```

**Note:** Using docker-compose will:
- ✅ Start Redis on port 6379
- ✅ Start Agent Memory Server on port 8088
- ✅ Configure networking between services
- ✅ Persist data in Docker volumes

If the Memory Server is not available, the notebook will skip memory-related demos but will still run.


---


### Automated Setup Check

Let's run the setup script to ensure all services are running properly.


In [1]:
# Check if services are running
import subprocess
import sys
from pathlib import Path

print("Checking if required services are running...\n")

# Check if Redis is running
try:
    result = subprocess.run(
        ["docker", "ps", "--filter", "name=redis", "--format", "{{.Names}}"],
        capture_output=True,
        text=True,
        timeout=5
    )
    if "redis" in result.stdout:
        print("✅ Redis is running")
    else:
        print("⚠️  Redis is not running. Start it with: docker-compose up -d")
except Exception as e:
    print(f"⚠️  Could not check Redis status: {e}")

# Check if Agent Memory Server is running
try:
    result = subprocess.run(
        ["docker", "ps", "--filter", "name=agent-memory", "--format", "{{.Names}}"],
        capture_output=True,
        text=True,
        timeout=5
    )
    if "agent-memory" in result.stdout or "memory" in result.stdout:
        print("✅ Agent Memory Server is running")
    else:
        print("⚠️  Agent Memory Server is not running. Start it with: docker-compose up -d")
except Exception as e:
    print(f"⚠️  Could not check Agent Memory Server status: {e}")

print("\nIf services are not running, start them with:")
print("  cd ../..")
print("  docker-compose up -d")

Checking if required services are running...

✅ Redis is running
✅ Agent Memory Server is running

If services are not running, start them with:
  cd ../..
  docker-compose up -d


---


### Install Dependencies

If you haven't already installed the reference-agent package, uncomment and run the following:


In [2]:
# Uncomment to install the project package
# %pip install -q -e ../..

# Uncomment to install agent-memory-client
# %pip install -q agent-memory-client

### Load Environment Variables

We'll load environment variables from the `.env` file in the `reference-agent` directory.

**Required variables:**
- `OPENAI_API_KEY` - Your OpenAI API key
- `REDIS_URL` - Redis connection URL (default: redis://localhost:6379)
- `AGENT_MEMORY_URL` - Agent Memory Server URL (default: http://localhost:8088)

If you haven't created the `.env` file yet, copy `.env.example` and add your OpenAI API key.


In [3]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables from project root directory
env_path = Path("../../.env")
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")

if not OPENAI_API_KEY:
    print(
        f"""❌ OPENAI_API_KEY not found!

    Please create a .env file at: {env_path.absolute()}

    With the following content:
    OPENAI_API_KEY=your_openai_api_key
    REDIS_URL=redis://localhost:6379
    AGENT_MEMORY_URL=http://localhost:8088
    """
    )
else:
    print("✅ Environment variables loaded")
    print(f"   REDIS_URL: {REDIS_URL}")
    print(f"   AGENT_MEMORY_URL: {AGENT_MEMORY_URL}")

✅ Environment variables loaded
   REDIS_URL: redis://localhost:6379
   AGENT_MEMORY_URL: http://localhost:8088


### Import Core Libraries

We'll import standard Python libraries and async support for our memory operations.


In [4]:
import asyncio
from datetime import datetime
from typing import Any, Dict, List, Optional

print("✅ Core libraries imported")

✅ Core libraries imported


### Import Section 2 Components

We're building on Section 2's RAG foundation, so we'll reuse the same components:
- `redis_config` - Redis connection and configuration
- `CourseManager` - Course search and management
- `StudentProfile` and other models - Data structures


In [5]:
from redis_context_course.course_manager import CourseManager
from redis_context_course.models import (
    Course,
    CourseFormat,
    DifficultyLevel,
    Semester,
    StudentProfile,
)

# Import Section 2 components
from redis_context_course.redis_config import redis_config

print("✅ Section 2 components imported")
print(f"   CourseManager: Available")
print(f"   Redis Config: Available")
print(f"   Models: Course, StudentProfile, etc.")

✅ Section 2 components imported
   CourseManager: Available
   Redis Config: Available
   Models: Course, StudentProfile, etc.


### Import LangChain Components

We'll use LangChain for LLM interaction and message handling.


In [6]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

print("✅ LangChain components imported")
print(f"   ChatOpenAI: Available")
print(f"   Message types: HumanMessage, SystemMessage, AIMessage")

✅ LangChain components imported
   ChatOpenAI: Available
   Message types: HumanMessage, SystemMessage, AIMessage


### Import Agent Memory Server Client

The Agent Memory Server provides production-ready memory management. If it's not available, we'll note that and continue with limited functionality.


In [7]:
# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import (
        ClientMemoryRecord,
        MemoryMessage,
        WorkingMemory,
    )

    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
    print("   MemoryAPIClient: Ready")
    print("   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("   Install with: pip install agent-memory-client")
    print("   Start server: See reference-agent/README.md")
    print("   Note: Some demos will be skipped")

✅ Agent Memory Server client available
   MemoryAPIClient: Ready
   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord


### What We Just Did

We've successfully set up our environment with all the necessary components:

**Imported:**
- ✅ Section 2 RAG components (`CourseManager`, `redis_config`, models)
- ✅ LangChain for LLM interaction
- ✅ Agent Memory Server client (if available)

**Why This Matters:**
- Building on Section 2's foundation (not starting from scratch)
- Agent Memory Server provides scalable, persistent memory
- Same Redis University domain for consistency

---

## 🔧 Initialize Components

Now let's initialize the components we'll use throughout this notebook.


### Initialize Course Manager

The `CourseManager` handles course search and retrieval, just like in Section 2.


In [8]:
# Initialize Course Manager
course_manager = CourseManager()

print("✅ Course Manager initialized")
print("   Ready to search and retrieve courses")

14:44:40 redisvl.index.index INFO   Index already exists, not overwriting.


✅ Course Manager initialized
   Ready to search and retrieve courses


### Initialize LLM

We'll use GPT-4o with temperature=0.0 for consistent, deterministic responses.


In [9]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

### Initialize Memory Client

If the Agent Memory Server is available, we'll initialize the memory client. This client handles both working memory (conversation history) and long-term memory (persistent facts).


In [10]:
# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL, default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print("✅ Memory Client initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
    print("   Ready for working memory and long-term memory operations")
else:
    memory_client = None
    print("⚠️  Memory Server not available")
    print("   Running with limited functionality")
    print("   Some demos will be skipped")

✅ Memory Client initialized
   Base URL: http://localhost:8088
   Namespace: redis_university
   Ready for working memory and long-term memory operations


### Create Sample Student Profile

We'll create a sample student profile to use throughout our demos. This follows the same pattern from Section 2.


In [11]:
# Create sample student profile
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
)

print("✅ Student profile created")
print(f"   Name: {sarah.name}")
print(f"   Major: {sarah.major}")
print(f"   Year: {sarah.year}")
print(f"   Interests: {', '.join(sarah.interests)}")
print(f"   Completed: {', '.join(sarah.completed_courses)}")
print(f"   Preferred Format: {sarah.preferred_format.value}")

✅ Student profile created
   Name: Sarah Chen
   Major: Computer Science
   Year: 2
   Interests: machine learning, data science, algorithms
   Completed: CS101, CS201
   Preferred Format: online


In [12]:
print("🎯 INITIALIZATION SUMMARY")
print(f"\n✅ Course Manager: Ready")
print(f"✅ LLM (GPT-4o): Ready")
print(
    f"{'✅' if MEMORY_SERVER_AVAILABLE else '⚠️ '} Memory Client: {'Ready' if MEMORY_SERVER_AVAILABLE else 'Not Available'}"
)
print(f"✅ Student Profile: {sarah.name}")

🎯 INITIALIZATION SUMMARY

✅ Course Manager: Ready
✅ LLM (GPT-4o): Ready
✅ Memory Client: Ready
✅ Student Profile: Sarah Chen


### Initialization Done
📋 What We're Building On:
-  Section 2's RAG foundation (CourseManager, redis_config)
-  Same StudentProfile model
-  Same Redis configuration

✨ What We're Adding:
-  Memory Client for conversation history
-  Working Memory for session context
-  Long-term Memory for persistent knowledge


---

## 📚 Part 1: Working Memory Fundamentals

### **What is Working Memory?**

Working memory stores **conversation messages** for the current session. It enables:

- ✅ **Reference resolution** - "it", "that course", "the one you mentioned"
- ✅ **Context continuity** - Each message builds on previous messages
- ✅ **Natural conversations** - Users don't repeat themselves

### **How It Works:**

```
Turn 1: Load working memory (empty) → Process query → Save messages
Turn 2: Load working memory (1 exchange) → Process query → Save messages
Turn 3: Load working memory (2 exchanges) → Process query → Save messages
```

Each turn has access to all previous messages in the session.

---

## 🧪 Hands-On: Working Memory in Action

Let's simulate a multi-turn conversation with working memory. We'll break this down step-by-step to see how working memory enables natural conversation flow.


### Setup: Create Session and Student IDs

Now that we have our components initialized, let's create session and student identifiers for our working memory demo.


In [13]:
# Setup for working memory demo
student_id = sarah.email.split("@")[0]  # "sarah.chen"
session_id = f"session_{student_id}_demo"

print("🎯 Working Memory Demo Setup")
print(f"   Student ID: {student_id}")
print(f"   Session ID: {session_id}")
print("   Ready to demonstrate multi-turn conversation")

🎯 Working Memory Demo Setup


   Student ID: sarah.chen
   Session ID: session_sarah.chen_demo
   Ready to demonstrate multi-turn conversation


### Turn 1: Initial Query

Let's start with a simple query about a course. This is the first turn, so working memory will be empty.

We'll break this down into clear steps:
1. We will use Memory Server
2. Load working memory (will be empty on first turn)
3. Search for the course
4. Generate a response
5. Save the conversation to working memory


#### Step 1: Set up the user query


In [14]:
# Check if Memory Server is available

print("=" * 80)
print("📍 TURN 1: User asks about a course")
print("=" * 80)

# Define the user's query
turn1_query = "Tell me about Data Structures and Algorithms"
print(f"\n👤 User: {turn1_query}")

📍 TURN 1: User asks about a course

👤 User: Tell me about Data Structures and Algorithms


#### Step 2: Load working memory

On the first turn, working memory will be empty since this is a new session.


In [15]:
# Load working memory (empty for first turn)
_, turn1_working_memory = await memory_client.get_or_create_working_memory(
    session_id=session_id, user_id=student_id, model_name="gpt-4o"
)

print(f"📊 Working Memory Status:")
print(f"   Messages in memory: {len(turn1_working_memory.messages)}")
print(
    f"   Status: {'Empty (first turn)' if len(turn1_working_memory.messages) == 0 else 'Has history'}"
)

14:44:41 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"


📊 Working Memory Status:
   Messages in memory: 20
   Status: Has history


#### Step 3: Search for the course

Use the course manager to search for courses matching the query.


In [16]:
print(f"\n🔍 Searching for courses...")
turn1_courses = await course_manager.search_courses(turn1_query, limit=1)

if turn1_courses:
    print(f"   Found {len(turn1_courses)} course(s)")

    # print the course details
    for course in turn1_courses:
        print(f"   - {course.course_code}: {course.title}")


🔍 Searching for courses...


14:44:42 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


   Found 1 course(s)
   - CS009: Data Structures and Algorithms


#### Step 4: Generate response using LLM

Use the LLM to generate a natural response based on the retrieved course information.

This follows the **RAG pattern**: Retrieve (done in Step 3) → Augment (add to context) → Generate (use LLM).


In [17]:
course = turn1_courses[0]

course_context = f"""Course Information:
- Code: {course.course_code}
- Title: {course.title}
- Description: {course.description}
- Prerequisites: {', '.join([p.course_code for p in course.prerequisites]) if course.prerequisites else 'None'}
- Credits: {course.credits}
"""

print(f"   Course context: {course_context}")

   Course context: Course Information:
- Code: CS009
- Title: Data Structures and Algorithms
- Description: Study of fundamental data structures and algorithms. Arrays, linked lists, trees, graphs, sorting, and searching.
- Prerequisites: CS001, CS001
- Credits: 4



In [18]:
# Build messages for LLM
turn1_messages = [
    SystemMessage(
        content="You are a helpful course advisor. Answer questions about courses based on the provided information."
    ),
    HumanMessage(content=f"{course_context}\n\nUser question: {turn1_query}"),
]

# Generate response using LLM
print(f"\n💭 Generating response using LLM...")
turn1_response = llm.invoke(turn1_messages).content

print(f"\n🤖 Agent: {turn1_response}")


💭 Generating response using LLM...


14:44:44 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: The course "Data Structures and Algorithms" (CS009) is a comprehensive study of fundamental data structures and algorithms. It covers essential topics such as arrays, linked lists, trees, graphs, sorting, and searching. This course is designed to provide a solid foundation in understanding how data can be organized and manipulated efficiently, which is crucial for solving complex computational problems.

To enroll in this course, you must have completed the prerequisite course CS001. The course is worth 4 credits, indicating a significant time commitment and depth of study. This course is essential for anyone looking to deepen their understanding of computer science and improve their problem-solving skills in programming.


#### Step 5: Save to working memory

Add both the user query and assistant response to working memory for future turns.


In [19]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn1_working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=turn1_query),
            MemoryMessage(role="assistant", content=turn1_response),
        ]
    )

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn1_working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    print(f"\n✅ Saved to working memory")
    print(f"   Messages now in memory: {len(turn1_working_memory.messages)}")

14:44:44 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



✅ Saved to working memory
   Messages now in memory: 22


### What Just Happened in Turn 1?

**Initial State:**
- Working memory was empty (first turn)
- No conversation history available

**Actions (RAG Pattern):**
1. **Retrieve:** Searched for Data Structures and Algorithms in the course database
2. **Augment:** Added course information to LLM context
3. **Generate:** LLM created a natural language response
4. **Save:** Stored conversation in working memory

**Result:**
- Working memory now contains 2 messages (1 user, 1 assistant)
- This history will be available for the next turn

**Key Insight:** Even the first turn uses the LLM to generate natural responses based on retrieved information.

---


### Turn 2: Follow-up with Pronoun Reference

Now let's ask a follow-up question using "its" - a pronoun that requires context from Turn 1.

We'll break this down into steps:
1. Set up the query with pronoun reference
2. Load working memory (now contains Turn 1)
3. Build context with conversation history
4. Generate response using LLM
5. Save to working memory


#### Step 1: Set up the query


In [20]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 2: User uses pronoun reference ('its')")
    print("=" * 80)

    turn2_query = "What are its prerequisites?"
    print(f"\n👤 User: {turn2_query}")
    print(f"   Note: 'its' refers to Data Structures and Algorithms from Turn 1")


📍 TURN 2: User uses pronoun reference ('its')

👤 User: What are its prerequisites?
   Note: 'its' refers to Data Structures and Algorithms from Turn 1


#### Step 2: Load working memory

This time, working memory will contain the conversation from Turn 1.


In [21]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (now has 1 exchange from Turn 1)
    _, turn2_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    print(f"\n📊 Working Memory Status:")
    print(f"   Messages in memory: {len(turn2_working_memory.messages)}")
    print(f"   Contains: Turn 1 conversation")

14:44:44 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"



📊 Working Memory Status:
   Messages in memory: 22
   Contains: Turn 1 conversation


#### Step 3: Build context with conversation history

To resolve the pronoun "its", we need to include the conversation history in the LLM context.


In [22]:
if MEMORY_SERVER_AVAILABLE:
    print(f"\n🔧 Building context with conversation history...")

    # Start with system message
    turn2_messages = [
        SystemMessage(
            content="You are a helpful course advisor. Use conversation history to resolve references like 'it', 'that course', etc."
        )
    ]

    # Add conversation history from working memory
    for msg in turn2_working_memory.messages:
        if msg.role == "user":
            turn2_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn2_messages.append(AIMessage(content=msg.content))

    # Add current query
    turn2_messages.append(HumanMessage(content=turn2_query))

    print(f"   Total messages in context: {len(turn2_messages)}")
    print(f"   Includes: System prompt + Turn 1 history + current query")


🔧 Building context with conversation history...
   Total messages in context: 24
   Includes: System prompt + Turn 1 history + current query


#### Step 4: Generate response using LLM

The LLM can now resolve "its" by looking at the conversation history.


In [23]:
if MEMORY_SERVER_AVAILABLE:
    print(f"\n💭 LLM resolving 'its' using conversation history...")
    turn2_response = llm.invoke(turn2_messages).content

    print(f"\n🤖 Agent: {turn2_response}")


💭 LLM resolving 'its' using conversation history...


14:44:47 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: The prerequisite for the "Data Structures and Algorithms" course (CS009) is CS001. This prerequisite ensures that students have a foundational understanding of computer science principles and basic programming skills, which are essential for tackling the more advanced topics covered in data structures and algorithms.


#### Step 5: Save to working memory

Add this turn's conversation to working memory for future turns.


In [24]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn2_working_memory.messages.extend(
        [
            MemoryMessage(role="user", content=turn2_query),
            MemoryMessage(role="assistant", content=turn2_response),
        ]
    )

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn2_working_memory,
        user_id=student_id,
        model_name="gpt-4o",
    )

    print(f"\n✅ Saved to working memory")
    print(f"   Messages now in memory: {len(turn2_working_memory.messages)}")

14:44:47 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"



✅ Saved to working memory
   Messages now in memory: 24


### What Just Happened in Turn 2?

**Initial State:**
- Working memory contained Turn 1 conversation (2 messages)
- User asked about "its prerequisites" - pronoun reference

**Actions:**
1. Loaded working memory with Turn 1 history
2. Built context including conversation history
3. LLM resolved "its" → Data Structures and Algorithms (from Turn 1)
4. Generated response about Data Structures and Algorithms's prerequisites
5. Saved updated conversation to working memory

**Result:**
- Working memory now contains 4 messages (2 exchanges)
- LLM successfully resolved pronoun reference using conversation history
- Natural conversation flow maintained

**Key Insight:** Without working memory, the LLM wouldn't know what "its" refers to!

---


### Turn 3: Another Follow-up

Let's ask one more follow-up question to demonstrate continued conversation continuity.


#### Step 1: Set up the query


In [25]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 3: User asks another follow-up")
    print("=" * 80)

    turn3_query = "Can I take it next semester?"
    print(f"\n👤 User: {turn3_query}")
    print(f"   Note: 'it' refers to Data Structures and Algorithms from Turn 1")


📍 TURN 3: User asks another follow-up

👤 User: Can I take it next semester?
   Note: 'it' refers to Data Structures and Algorithms from Turn 1


#### Step 2: Load working memory with full conversation history


In [26]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (now has 2 exchanges)
    _, turn3_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id, user_id=student_id, model_name="gpt-4o"
    )

    print(f"\n📊 Working Memory Status:")
    print(f"   Messages in memory: {len(turn3_working_memory.messages)}")
    print(f"   Contains: Turns 1 and 2")

14:44:47 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"



📊 Working Memory Status:
   Messages in memory: 24
   Contains: Turns 1 and 2


#### Step 3: Build context and generate response


In [27]:
if MEMORY_SERVER_AVAILABLE:
    # Build context with full conversation history
    turn3_messages = [
        SystemMessage(
            content="You are a helpful course advisor. Use conversation history to resolve references."
        )
    ]

    for msg in turn3_working_memory.messages:
        if msg.role == "user":
            turn3_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn3_messages.append(AIMessage(content=msg.content))

    turn3_messages.append(HumanMessage(content=turn3_query))

    print(f"   Total messages in context: {len(turn3_messages)}")

    # Generate response
    turn3_response = llm.invoke(turn3_messages).content

    print(f"\n🤖 Agent: {turn3_response}")

   Total messages in context: 26


14:44:50 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 Agent: Yes, you can take the "Data Structures and Algorithms" course (CS009) next semester. It's important to ensure that you have completed the prerequisite course, CS001, before enrolling. If you meet the prerequisite requirement, you should be able to register for the course when enrollment opens for the next semester. Be sure to check with your academic advisor or the course catalog for specific enrollment dates and any additional requirements.




✅ DEMO COMPLETE: Working memory enabled natural conversation flow!

---
### Working Memory Demo Summary

Let's review what we just demonstrated across three conversation turns.

## 🎯 Working Memory Demo Summary
### 📊 What Happened:
**Turn 1:** 'Tell me about Data Structures and Algorithms'
- Working memory: empty (first turn)
- Stored query and response

**Turn 2:** 'What are its prerequisites?'
- Working memory: 1 exchange (Turn 1)
- LLM resolved 'its' → Data Structures and Algorithms using history
- Generated accurate response

**Turn 3:** 'Can I take it next semester?'
- Working memory: 2 exchanges (Turns 1-2)
- LLM resolved 'it' → Data Structures and Algorithms using history
- Maintained conversation continuity

#### ✅ Key Benefits:
- Natural conversation flow
- Pronoun reference resolution
- No need to repeat context
- Seamless user experience

#### ❌ Without Working Memory:
- 'What are its prerequisites?' → 'What is its?' Or "General information without data from the LLM's training"
- Each query is isolated
- User must repeat context every time

### Key Insight: Conversation Context Type

Working memory provides the **Conversation Context** - the third context type from Section 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile and preferences (dynamic, user-specific)
3. **Conversation Context** - Working memory (dynamic, session-specific) ← **We just demonstrated this!**
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Without working memory, we only had 3 context types. Now we have all 4!


---
# 📚 Part 2: Long-term Memory for Context Engineering

## What is Long-term Memory?

Long-term memory enables AI agents to store **persistent knowledge** across sessions—including user preferences, domain facts, business rules, and system configuration. This is crucial for context engineering because it allows agents to:

- **Personalize** interactions by remembering user-specific preferences and history
- **Apply domain knowledge** consistently (prerequisites, policies, regulations)
- **Maintain organizational context** (business rules, schedules, procedures)
- **Search efficiently** using semantic vector search across all knowledge types

Long-term memory is a flexible storage mechanism: user-scoped memories enable personalization ("Student prefers online courses"), while application-scoped memories provide consistent behavior for everyone ("CS401 requires CS201", "Registration opens 2 weeks before semester").

### How It Works

```
Session 1: User shares preferences → Store in long-term memory
Session 2: User asks for recommendations → Search memory → Personalized response
Session 3: User updates preferences → Update memory accordingly
```

---

## Three Types of Long-term Memory

The Agent Memory Server supports three distinct memory types, each optimized for different kinds of information:

### 1. Semantic Memory - Facts and Knowledge

**Purpose:** Store timeless facts, preferences, and knowledge independent of when they were learned. Can be user-scoped (personalization) or application-scoped (domain knowledge).

**User-Scoped Examples:**
- "Student's major is Computer Science"
- "Student prefers online courses"
- "Student wants to graduate in Spring 2026"
- "Student is interested in machine learning"

**Application-Scoped Examples:**
- "CS401 requires CS201 and MATH301 as prerequisites"
- "Online courses have asynchronous discussion forums"
- "Academic advisors are available Monday-Friday 9am-5pm"
- "Maximum file upload size for assignments is 50MB"

**When to use:** Information that remains true regardless of time context, whether user-specific or universally applicable.

---

### 2. Episodic Memory - Events and Experiences

**Purpose:** Store time-bound events and experiences where sequence matters.

**Examples:**
- "Student enrolled in CS101 on 2024-09-15"
- "Student completed CS101 with grade A on 2024-12-10"
- "Student asked about machine learning courses on 2024-09-20"

**When to use:** Timeline-based information where timing or sequence is important.

---

### 3. Message Memory - Context-Rich Conversations

**Purpose:** Store full conversation snippets where complete context is crucial.

**Examples:**
- Detailed career planning discussion with nuanced advice
- Professor's specific guidance about research opportunities
- Student's explanation of personal learning challenges

**When to use:** When summary would lose important nuance, tone, or exact wording.

**⚠️ Use sparingly** - Message memories are token-expensive!

---

## 🎯 Choosing the Right Memory Type

### Decision Framework

**Ask yourself these questions:**

1. **Can you extract a simple fact?** → Use **Semantic**
2. **Does timing matter?** → Use **Episodic**
3. **Is full context crucial?** → Use **Message** (rarely)

**Default strategy: Prefer Semantic** - they're compact, searchable, and efficient.

---

### Quick Reference Table

| Information Type | Memory Type | Example |
|-----------------|-------------|----------|
| Preference | Semantic | "Prefers morning classes" |
| Fact | Semantic | "Major is Computer Science" |
| Goal | Semantic | "Wants to graduate in 2026" |
| Event | Episodic | "Enrolled in CS401 on 2024-09-15" |
| Timeline | Episodic | "Completed CS101, then CS201" |
| Complex discussion | Message | [Full career planning conversation] |
| Nuanced advice | Message | [Professor's detailed guidance] |

---

## Examples: Right vs. Wrong Choices

### Scenario 1: Student States Preference

**User says:** "I prefer online courses because I work during the day."

❌ **Wrong - Message memory (too verbose):**
```python
memory = "Student said: 'I prefer online courses because I work during the day.'"
```

✅ **Right - Semantic memories (extracted facts):**
```python
memory1 = "Student prefers online courses"
memory2 = "Student works during the day"
```

**Why:** Simple facts don't need verbatim storage.

---

### Scenario 2: Course Completion

**User says:** "I just finished CS101 last week!"

❌ **Wrong - Semantic (loses temporal context):**
```python
memory = "Student completed CS101"
```

✅ **Right - Episodic (preserves timeline):**
```python
memory = "Student completed CS101 on 2024-10-20"
```

**Why:** Timeline matters for prerequisites and future planning.

---

### Scenario 3: Complex Career Advice

**Context:** 20-message discussion about career path including nuanced advice about research vs. industry, application timing, and specific companies to target.

❌ **Wrong - Semantic (loses too much context):**
```python
memory = "Student discussed career planning"
```

✅ **Right - Message memory (preserves full context):**
```python
memory = [Full conversation thread with all nuance]
```

**Why:** Details and context are critical; summary would be inadequate.

---

## Key Takeaways

- **Most memories should be semantic** - efficient and searchable
- **Use episodic when sequence matters** - track progress and timeline
- **Use message rarely** - only when context cannot be summarized
- **Effective memory selection improves personalization** and reduces token usage

---

## 🧪 Hands-On: Long-term Memory in Action

Let's put these concepts into practice with code examples...

### Setup: Student ID for Long-term Memory

Long-term memories are user-scoped, so we need a student ID.


In [28]:
# Setup for long-term memory demo
lt_student_id = "sarah_chen"

print("🎯 Long-term Memory Demo Setup")
print(f"   Student ID: {lt_student_id}")
print("   Ready to store and search persistent memories")

🎯 Long-term Memory Demo Setup
   Student ID: sarah_chen
   Ready to store and search persistent memories


### Step 1: Store Semantic Memories (Facts)

Semantic memories are timeless facts about the student. Let's store several facts about Sarah's preferences and academic status.


In [29]:
print("=" * 80)
print("📍 STEP 1: Storing Semantic Memories (Facts)")
print("=" * 80)

# Define semantic memories (timeless facts)
semantic_memories = [
    "Student prefers online courses over in-person classes",
    "Student's major is Computer Science with focus on AI/ML",
    "Student wants to graduate in Spring 2026",
    "Student prefers morning classes, no classes on Fridays",
    "Student has completed Introduction to Programming and Data Structures",
    "Student is currently taking Linear Algebra",
]
print(f"\n📝 Storing {len(semantic_memories)} semantic memories...")

# Store each semantic memory
for memory_text in semantic_memories:
    memory_record = ClientMemoryRecord(
        text=memory_text,
        user_id=lt_student_id,
        memory_type="semantic",
        topics=["preferences", "academic_info"],
    )
await memory_client.create_long_term_memory([memory_record])
print(f"   ✅ {memory_text}")

print(f"\n✅ Stored {len(semantic_memories)} semantic memories")
print("   Memory type: semantic (timeless facts)")
print("   Topics: preferences, academic_info")

📍 STEP 1: Storing Semantic Memories (Facts)

📝 Storing 6 semantic memories...
14:44:50 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student is currently taking Linear Algebra

✅ Stored 6 semantic memories
   Memory type: semantic (timeless facts)
   Topics: preferences, academic_info


### What We Just Did: Semantic Memories

**Stored 6 semantic memories:**
- Student preferences (online courses, morning classes)
- Academic information (major, graduation date)
- Course history (completed, current)

**Why semantic?**
- These are timeless facts
- No specific date/time context needed
- Compact and efficient

**How they're stored:**
- Vector-indexed for semantic search
- Tagged with topics for organization
- Automatically deduplicated

---


### Step 2: Store Episodic Memories (Events)

Episodic memories are time-bound events. Let's store some events from Sarah's academic timeline.


In [30]:
print("\n" + "=" * 80)
print("📍 STEP 2: Storing Episodic Memories (Events)")
print("=" * 80)

# Define episodic memories (time-bound events)
episodic_memories = [
    "Student enrolled in Introduction to Programming on 2024-09-01",
    "Student completed Introduction to Programming with grade A on 2024-12-15",
    "Student asked about machine learning courses on 2024-09-20",
]

print(f"\n📝 Storing {len(episodic_memories)} episodic memories...")

# Store each episodic memory
for memory_text in episodic_memories:
    memory_record = ClientMemoryRecord(
        text=memory_text,
        user_id=lt_student_id,
        memory_type="episodic",
        topics=["enrollment", "courses"],
    )
    await memory_client.create_long_term_memory([memory_record])
    print(f"   ✅ {memory_text}")

print(f"\n✅ Stored {len(episodic_memories)} episodic memories")
print("   Memory type: episodic (time-bound events)")
print("   Topics: enrollment, courses")


📍 STEP 2: Storing Episodic Memories (Events)

📝 Storing 3 episodic memories...
14:44:50 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student enrolled in Introduction to Programming on 2024-09-01
14:44:50 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student completed Introduction to Programming with grade A on 2024-12-15
14:44:50 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student asked about machine learning courses on 2024-09-20

✅ Stored 3 episodic memories
   Memory type: episodic (time-bound events)
   Topics: enrollment, courses


### What We Just Did: Episodic Memories

**Stored 3 episodic memories:**
- Enrollment event (Introduction to Programming on 2024-09-01)
- Completion event (Introduction to Programming with grade A on 2024-12-15)
- Interaction event (asked about ML courses on 2024-09-20)

**Why episodic?**
- These are time-bound events
- Timing and sequence matter
- Captures academic timeline

**Difference from semantic:**
- Semantic: "Student has completed Introduction to Programming" (timeless fact)
- Episodic: "Student completed Introduction to Programming with grade A on 2024-12-15" (specific event)

---


### Step 3: Search Long-term Memory

Now let's search our long-term memories using natural language queries. The system will use semantic search to find relevant memories.


#### Query 1: What does the student prefer?


In [31]:
if MEMORY_SERVER_AVAILABLE:
    from agent_memory_client.filters import UserId

    print("\n" + "=" * 80)
    print("📍 STEP 3: Searching Long-term Memory")
    print("=" * 80)

    search_query_1 = "What does the student prefer?"
    print(f"\n🔍 Query: '{search_query_1}'")

    search_results_1 = await memory_client.search_long_term_memory(
        text=search_query_1, user_id=UserId(eq=lt_student_id), limit=3
    )

    if search_results_1.memories:
        print(f"   📚 Found {len(search_results_1.memories)} relevant memories:")
        for i, memory in enumerate(search_results_1.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")


📍 STEP 3: Searching Long-term Memory

🔍 Query: 'What does the student prefer?'


14:44:51 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   📚 Found 3 relevant memories:
      1. The student prefers online courses over in-person classes.
      2. Student prefers morning classes
      3. Student prefers morning classes, no classes on Fridays


#### Query 2: What courses has the student completed?


In [32]:
if MEMORY_SERVER_AVAILABLE:
    search_query_2 = "What courses has the student completed?"
    print(f"\n🔍 Query: '{search_query_2}'")

    search_results_2 = await memory_client.search_long_term_memory(
        text=search_query_2, user_id=UserId(eq=lt_student_id), limit=5
    )

    if search_results_2.memories:
        print(f"   📚 Found {len(search_results_2.memories)} relevant memories:")
        for i, memory in enumerate(search_results_2.memories[:5], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")


🔍 Query: 'What courses has the student completed?'


14:44:51 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   📚 Found 5 relevant memories:
      1. Student has completed Introduction to Programming and Data Structures
      2. Student completed Introduction to Programming with grade A on 2024-12-15
      3. Student's major is Computer Science
      4. Student is currently taking Linear Algebra
      5. Student asked about machine learning courses on 2024-09-20


#### Query 3: What is the student's major?


In [33]:
if MEMORY_SERVER_AVAILABLE:
    search_query_3 = "What is the student's major?"
    print(f"\n🔍 Query: '{search_query_3}'")

    search_results_3 = await memory_client.search_long_term_memory(
        text=search_query_3, user_id=UserId(eq=lt_student_id), limit=3
    )

    if search_results_3.memories:
        print(f"   📚 Found {len(search_results_3.memories)} relevant memories:")
        for i, memory in enumerate(search_results_3.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!")
    print("=" * 80)
else:
    print("⚠️  Memory Server not available. Skipping demo.")


🔍 Query: 'What is the student's major?'


14:44:52 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   📚 Found 3 relevant memories:
      1. Student's major is Computer Science
      2. Student's major is Computer Science with focus on AI/ML
      3. Student wants to graduate in Spring 2026

✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!


### Long-term Memory Demo Summary

Let's review what we demonstrated with long-term memory.


In [34]:
print("=" * 80)
print("🎯 LONG-TERM MEMORY DEMO SUMMARY")
print("=" * 80)
print("\n📊 What We Did:")
print("   Step 1: Stored 6 semantic memories (facts)")
print("           → Student preferences, major, graduation date")
print("           → Tagged with topics: preferences, academic_info")
print("\n   Step 2: Stored 3 episodic memories (events)")
print("           → Enrollment, completion, interaction events")
print("           → Tagged with topics: enrollment, courses")
print("\n   Step 3: Searched long-term memory")
print("           → Used natural language queries")
print("           → Semantic search found relevant memories")
print("           → No exact keyword matching needed")
print("\n✅ Key Benefits:")
print("   • Persistent knowledge across sessions")
print("   • Semantic search (not keyword matching)")
print("   • Automatic deduplication")
print("   • Topic-based organization")
print("\n💡 Key Insight:")
print("   Long-term memory enables personalization and knowledge")
print("   accumulation across sessions. It's the foundation for")
print("   building agents that remember and learn from users.")
print("=" * 80)

🎯 LONG-TERM MEMORY DEMO SUMMARY

📊 What We Did:
   Step 1: Stored 6 semantic memories (facts)
           → Student preferences, major, graduation date
           → Tagged with topics: preferences, academic_info

   Step 2: Stored 3 episodic memories (events)
           → Enrollment, completion, interaction events
           → Tagged with topics: enrollment, courses

   Step 3: Searched long-term memory
           → Used natural language queries
           → Semantic search found relevant memories
           → No exact keyword matching needed

✅ Key Benefits:
   • Persistent knowledge across sessions
   • Semantic search (not keyword matching)
   • Automatic deduplication
   • Topic-based organization

💡 Key Insight:
   Long-term memory enables personalization and knowledge
   accumulation across sessions. It's the foundation for
   building agents that remember and learn from users.


### Key Insight: User Context Type

Long-term memory provides part of the **User Context** - the second context type from Section 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile + long-term memories (dynamic, user-specific) ← **Long-term memories contribute here!**
3. **Conversation Context** - Working memory (dynamic, session-specific)
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Long-term memories enhance User Context by adding persistent knowledge about the user's preferences, history, and goals.

---

## 🏷️ Advanced: Topics and Filtering

Topics help organize and filter memories. Let's explore how to use them effectively.


### Step 1: Store memories with topics


In [35]:
if MEMORY_SERVER_AVAILABLE:
    topics_student_id = "sarah_chen"

    print("=" * 80)
    print("🏷️  TOPICS AND FILTERING DEMO")
    print("=" * 80)

    print("\n📍 Storing Memories with Topics")
    print("-" * 80)

    # Define memories with their topics
    memories_with_topics = [
        ("Student prefers online courses", ["preferences", "course_format"]),
        ("Student's major is Computer Science", ["academic_info", "major"]),
        ("Student wants to graduate in Spring 2026", ["goals", "graduation"]),
        ("Student prefers morning classes", ["preferences", "schedule"]),
    ]

    # Store each memory
    for memory_text, topics in memories_with_topics:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=topics_student_id,
            memory_type="semantic",
            topics=topics,
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")
        print(f"      Topics: {', '.join(topics)}")

🏷️  TOPICS AND FILTERING DEMO

📍 Storing Memories with Topics
--------------------------------------------------------------------------------
14:44:52 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers online courses
      Topics: preferences, course_format
14:44:52 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student's major is Computer Science
      Topics: academic_info, major
14:44:52 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student wants to graduate in Spring 2026
      Topics: goals, graduation
14:44:52 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Student prefers morning classes
      Topics: preferences, schedule


### Step 2: Filter memories by type


In [36]:
if MEMORY_SERVER_AVAILABLE:
    print("\n📍 Filtering by Memory Type: Semantic")
    print("-" * 80)

    from agent_memory_client.filters import MemoryType, UserId

    # Search for all semantic memories
    results = await memory_client.search_long_term_memory(
        text="",  # Empty query returns all
        user_id=UserId(eq=topics_student_id),
        memory_type=MemoryType(eq="semantic"),
        limit=10,
    )

    print(f"   Found {len(results.memories)} semantic memories:")
    for i, memory in enumerate(results.memories[:5], 1):
        topics_str = ", ".join(memory.topics) if memory.topics else "none"
        print(f"   {i}. {memory.text}")
        print(f"      Topics: {topics_str}")

    print("\n" + "=" * 80)
    print("✅ Topics enable organized, filterable memory management!")
    print("=" * 80)


📍 Filtering by Memory Type: Semantic
--------------------------------------------------------------------------------


14:44:53 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"


   Found 9 semantic memories:
   1. The student prefers online courses over in-person classes.
      Topics: preferences, course_format, academic_info
   2. Student is currently taking Linear Algebra
      Topics: preferences, academic_info
   3. Student's major is Computer Science
      Topics: academic_info, major
   4. Student prefers morning classes
      Topics: preferences, schedule
   5. Student is interested in machine learning and AI
      Topics: interests, AI

✅ Topics enable organized, filterable memory management!


### 🎯 Why Topics Matter

**Organization:**
- Group related memories together
- Easy to find memories by category

**Filtering:**
- Search within specific topics
- Filter by memory type (semantic, episodic, message)

**Best Practices:**
- Use consistent topic names
- Keep topics broad enough to be useful
- Common topics: `preferences`, `academic_info`, `goals`, `schedule`, `courses`

---

## 🔄 Cross-Session Memory Persistence

Let's verify that memories persist across sessions.


### Step 1: Session 1 - Store memories


In [37]:
if MEMORY_SERVER_AVAILABLE:
    cross_session_student_id = "sarah_chen"

    print("=" * 80)
    print("🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO")
    print("=" * 80)

    print("\n📍 SESSION 1: Storing Memories")
    print("-" * 80)

    memory_record = ClientMemoryRecord(
        text="Student is interested in machine learning and AI",
        user_id=cross_session_student_id,
        memory_type="semantic",
        topics=["interests", "AI"],
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored: Student is interested in machine learning and AI")

🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO

📍 SESSION 1: Storing Memories
--------------------------------------------------------------------------------
14:44:53 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Stored: Student is interested in machine learning and AI


### Step 2: Session 2 - Create new client and retrieve memories

Simulate a new session by creating a new memory client.


In [38]:
# Search for memories from the new session
from agent_memory_client.filters import UserId

if MEMORY_SERVER_AVAILABLE:
    print("\n📍 SESSION 2: New Session, Same Student")
    print("-" * 80)

    # Create a new memory client (simulating a new session)
    new_session_config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university",
    )
    new_session_client = MemoryAPIClient(config=new_session_config)

    print("   🔄 New session started for the same student")

    print("\n   🔍 Searching: 'What are the student's interests?'")
    cross_session_results = await new_session_client.search_long_term_memory(
        text="What are the student's interests?",
        user_id=UserId(eq=cross_session_student_id),
        limit=3,
    )

    if cross_session_results.memories:
        print(f"\n   ✅ Memories accessible from new session:")
        for i, memory in enumerate(cross_session_results.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist across sessions!")
    print("=" * 80)


📍 SESSION 2: New Session, Same Student
--------------------------------------------------------------------------------
   🔄 New session started for the same student

   🔍 Searching: 'What are the student's interests?'


14:44:53 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"



   ✅ Memories accessible from new session:
      1. Student is interested in machine learning and AI
      2. Student's major is Computer Science
      3. Student's major is Computer Science with focus on AI/ML

✅ Long-term memories persist across sessions!


### 🎯 Cross-Session Persistence

**What We Demonstrated:**
- **Session 1:** Stored memories about student interests
- **Session 2:** Created new client (simulating new session)
- **Result:** Memories from Session 1 are accessible in Session 2

**Why This Matters:**
- Users don't have to repeat themselves
- Personalization works across days, weeks, months
- Knowledge accumulates over time

**Contrast with Working Memory:**
- Working memory: Session-scoped (persists within the session, like ChatGPT conversations)
- Long-term memory: User-scoped (persists across all sessions indefinitely)

---

## 🔗 What's Next: Memory-Enhanced RAG and Agents

You've learned the fundamentals of memory architecture! Now it's time to put it all together.

### **Next Notebook: `02_combining_memory_with_retrieved_context.ipynb`**

In the next notebook, you'll:

1. **Build** a complete memory-enhanced RAG system
   - Integrate working memory + long-term memory + RAG
   - Combine all four context types
   - Show clear before/after comparisons

2. **Convert** to LangGraph agent (Part 2, separate notebook)
   - Add state management
   - Improve control flow
   - Prepare for Section 4 (tools and advanced capabilities)

**Why Continue?**
- See memory in action with real conversations
- Learn how to build production-ready agents
- Prepare for Section 4 (adding tools like enrollment, scheduling)

**📚 Continue to:** `02_combining_memory_with_retrieved_context.ipynb`

## ⏰ Memory Lifecycle & Persistence

Understanding how working memory and long-term memory persist is crucial for building reliable systems.

### **Working Memory Persistence**

**How it works:** Just like ChatGPT or Claude conversations

**What this means:**
- When you return to a conversation, the working memory is still there
- The conversation doesn't disappear when you close the tab
- Full conversation history remains accessible within the session
- **Backend optimization:** TTL for storage management (not user-facing)

**User Experience:**

```
Day 1, 10:00 AM - User starts conversation
Day 1, 10:25 AM - User closes browser
    ↓
[User returns later]
    ↓
Day 1, 3:00 PM - User reopens conversation
                 → Working memory still there ✅
                 → Conversation continues naturally ✅
```

**The Real Challenge: Context Window Limits**

Working memory doesn't "expire" - but it can grow too large:
- LLMs have context window limits (e.g., 128K tokens for GPT-4)
- Long conversations eventually exceed these limits
- **Solution:** Compression strategies (covered in Notebook 03)

### **Long-term Memory Persistence**

**Lifetime:** Indefinite (until manually deleted)

**What this means:**
- Long-term memories never expire automatically
- Accessible across all sessions, forever
- Must be explicitly deleted if no longer needed

### **Why This Design?**

**Working Memory (Session-Persistent):**
- Stores full conversation history for the session
- Persists when you return to the conversation (like ChatGPT)
- **Challenge:** Can grow too large for context window
- **Solution:** Compression strategies (Notebook 03)

**Long-term Memory (Cross-Session Persistent):**
- Important facts extracted from conversations
- User preferences don't expire
- Knowledge accumulates over time
- Enables true personalization across sessions

### **Important Implications**

**1. Automatic Extraction to Long-term Memory**

Important facts from conversations are automatically extracted to long-term memory.

**Good news:** Agent Memory Server does this automatically in the background!

**2. Long-term Memories are Permanent**

Once stored, long-term memories persist indefinitely. Be thoughtful about what you store.

**3. Cross-Session Behavior**

```
Session 1 (Day 1):
- User: "I'm interested in machine learning"
- Working memory: Stores full conversation
- Long-term memory: Extracts "Student interested in machine learning"

[User starts a NEW session on Day 3]

Session 2 (Day 3):
- Working memory: NEW session, starts empty ✅
- Long-term memory: Still has "Student interested in machine learning" ✅
- Agent retrieves long-term memory for personalization ✅
- Agent makes relevant recommendations ✅
```

**Key Distinction:**
- **Same session:** Working memory persists (like returning to a ChatGPT conversation)
- **New session:** Working memory starts fresh, but long-term memories are available

### **Practical Multi-Day Conversation Example**


In [39]:
# Multi-Day Conversation Simulation
from agent_memory_client.filters import UserId


async def multi_day_simulation():
    """Simulate conversations across multiple days"""

    student_id = "sarah_chen"

    print("=" * 80)
    print("⏰ MULTI-DAY CONVERSATION SIMULATION")
    print("=" * 80)

    # Day 1: Initial conversation
    print("\n📅 DAY 1: Initial Conversation")
    print("-" * 80)

    session_1 = f"session_{student_id}_day1"
    text = "Student is preparing for a career in AI research"
    print(f"\nText: {text}\n")
    # Store a fact in long-term memory
    memory_record = ClientMemoryRecord(
        text=text,
        user_id=student_id,
        memory_type="semantic",
        topics=["career", "goals"],
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored in long-term memory: Career goal (AI research)")

    # Simulate working memory (would normally be conversation)
    print("   💬 Working memory: Active for session_day1")
    print("   📝 Note: If user returns to THIS session, working memory persists")

    # Day 3: NEW conversation (different session)
    print("\n📅 DAY 3: NEW Conversation (different session)")
    print("-" * 80)

    session_2 = f"session_{student_id}_day3"

    print("   🆕 Working memory: NEW session, starts empty")
    print("   ✅ Long-term memory: Still available across all sessions")
    text2 = "What are the student's career goals?"
    print(f"\nText: {text2}\n")

    # Search long-term memory
    results = await memory_client.search_long_term_memory(
        text=text2, user_id=UserId(eq=student_id), limit=3
    )

    if results.memories:
        print("\n   🔍 Retrieved from long-term memory:")
        for memory in results.memories[:3]:
            print(f"      • {memory.text}")
        print("\n   ✅ Agent can still personalize recommendations!")

    print("\n" + "=" * 80)
    print(
        "✅ Long-term memories persist across sessions, working memory is session-scoped"
    )
    print("=" * 80)


# Run the simulation
await multi_day_simulation()

⏰ MULTI-DAY CONVERSATION SIMULATION

📅 DAY 1: Initial Conversation
--------------------------------------------------------------------------------

Text: Student is preparing for a career in AI research



14:44:53 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/ "HTTP/1.1 200 OK"


   ✅ Stored in long-term memory: Career goal (AI research)
   💬 Working memory: Active for session_day1
   📝 Note: If user returns to THIS session, working memory persists

📅 DAY 3: NEW Conversation (different session)
--------------------------------------------------------------------------------
   🆕 Working memory: NEW session, starts empty
   ✅ Long-term memory: Still available across all sessions

Text: What are the student's career goals?



14:44:53 httpx INFO   HTTP Request: POST http://localhost:8088/v1/long-term-memory/search?optimize_query=false "HTTP/1.1 200 OK"



   🔍 Retrieved from long-term memory:
      • Student wants to graduate in Spring 2026
      • Student's major is Computer Science
      • Student's major is Computer Science with focus on AI/ML

   ✅ Agent can still personalize recommendations!

✅ Long-term memories persist across sessions, working memory is session-scoped


### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Understand Session Management**
- Working memory persists within a session
- New sessions start with empty working memory
- Important facts should be in long-term memory for cross-session access
- Consider providing ways to resume or load previous session context

**5. Plan for Context Window Limits**
- Working memory doesn't expire, but can grow too large
- LLMs have context window limits (e.g., 128K tokens)
- Use compression strategies when conversations get long (covered in Notebook 03)
- Monitor token usage in long conversations

**6. Test Cross-Session Behavior**
- Verify long-term memories are accessible across sessions
- Test both same-session returns and new-session starts
- Ensure personalization works in both scenarios

---


## 🧠 Memory Extraction Strategies

The Agent Memory Server automatically extracts important information from conversations and stores it in long-term memory. Understanding **how** this extraction works helps you choose the right strategy for your use case.


### How Memory Extraction Works

**Key Distinction:**
- **Working Memory:** Stores raw conversation messages (user/assistant exchanges)
- **Long-term Memory:** Stores extracted facts, summaries, or preferences

**The Question:** When promoting information from working memory to long-term memory, should we extract:
- Individual discrete facts? ("User prefers online courses")
- A summary of the conversation? ("User discussed course preferences...")
- User preferences specifically? ("User prefers email notifications")
- Custom domain-specific information?

This is where **memory extraction strategies** come in.


### Available Strategies

The Agent Memory Server supports four memory extraction strategies that determine how memories are created:

#### **1. Discrete Strategy (Default)** ✅

**Purpose:** Extract individual facts and preferences from conversations

**Best For:** General-purpose memory extraction, factual information, user preferences

**Example Input (Conversation):**
```
User: "I'm a Computer Science major interested in machine learning. I prefer online courses."
```

**Example Output (Long-term Memories):**
```json
[
  {
    "type": "semantic",
    "text": "User's major is Computer Science",
    "topics": ["education", "major"],
    "entities": ["Computer Science"]
  },
  {
    "type": "semantic",
    "text": "User interested in machine learning",
    "topics": ["interests", "technology"],
    "entities": ["machine learning"]
  },
  {
    "type": "semantic",
    "text": "User prefers online courses",
    "topics": ["preferences", "learning"],
    "entities": ["online courses"]
  }
]
```

**When to Use:**
- ✅ Most agent interactions (default choice)
- ✅ When you want searchable individual facts
- ✅ When facts should be independently retrievable
- ✅ Building knowledge graphs or fact databases

---

#### **2. Summary Strategy**

**Purpose:** Create concise summaries of entire conversations instead of extracting discrete facts

**Best For:** Long conversations, meeting notes, comprehensive context preservation

**Example Input (Same Conversation):**
```
User: "I'm a Computer Science major interested in machine learning. I prefer online courses."
```

**Example Output (Long-term Memory):**
```json
{
  "type": "semantic",
  "text": "User is a Computer Science major with interest in machine learning, preferring online course formats for their studies.",
  "topics": ["education", "preferences", "technology"],
  "entities": ["Computer Science", "machine learning", "online courses"]
}
```

**When to Use:**
- ✅ Long consultations or advising sessions
- ✅ Meeting notes or session summaries
- ✅ When context of entire conversation matters
- ✅ Reducing storage while preserving conversational context

---

#### **3. Preferences Strategy**

**Purpose:** Focus specifically on extracting user preferences and personal characteristics

**Best For:** Personalization systems, user profile building, preference learning

**Example Output:**
```json
{
  "type": "semantic",
  "text": "User prefers online courses over in-person instruction",
  "topics": ["preferences", "learning_style"],
  "entities": ["online courses", "in-person"]
}
```

**When to Use:**
- ✅ User onboarding flows
- ✅ Building user profiles
- ✅ Personalization-focused applications
- ✅ Preference learning systems

---

#### **4. Custom Strategy**

**Purpose:** Use domain-specific extraction prompts for specialized needs

**Best For:** Domain-specific extraction (technical, legal, medical), specialized workflows

**Security Note:** ⚠️ Custom prompts require validation to prevent prompt injection attacks. See the [Security Guide](https://redis.github.io/agent-memory-server/security/) for details.

**When to Use:**
- ✅ Specialized domains (legal, medical, technical)
- ✅ Custom extraction logic needed
- ✅ Domain-specific memory structures

---


### Strategy Comparison

| Strategy | Output Type | Use Case | Example |
|----------|------------|----------|---------|
| **Discrete** | Individual facts | General agents | "User's major is Computer Science" |
| **Summary** | Conversation summary | Long sessions | "User discussed CS major, interested in ML courses..." |
| **Preferences** | User preferences | Personalization | "User prefers online courses over in-person" |
| **Custom** | Domain-specific | Specialized domains | Custom extraction logic |


### Default Behavior in This Course

**In this course, we use the Discrete Strategy (default)** because:

✅ **Works well for course advising conversations**
- Students ask specific questions
- Facts are independently useful
- Each fact can be searched separately

✅ **Creates searchable individual facts**
- "User's major is Computer Science"
- "User completed RU101"
- "User interested in machine learning"

✅ **Balances detail with storage efficiency**
- Not too granular (every sentence)
- Not too broad (entire conversations)
- Just right for Q&A interactions

✅ **No configuration required**
- Default behavior
- Works out of the box
- Production-ready


### When Would You Use Different Strategies?

**Scenario 1: Long Academic Advising Session (Summary Strategy)**

```
Student has 30-minute conversation discussing:
- Academic goals and graduation timeline
- Career aspirations and internship plans
- Course preferences and learning style
- Schedule constraints and work commitments
- Extracurricular interests
```

**Discrete Strategy:** Extracts 20+ individual facts
- "User wants to graduate Spring 2026"
- "User interested in tech startup internship"
- "User prefers online courses"
- ... (17 more facts)

**Summary Strategy:** Creates 1-2 comprehensive summaries
- "Student discussed academic planning for Spring 2026 graduation, expressing strong interest in ML/AI courses and tech startup internships. Prefers online format due to part-time work commitments. Interested in vector databases and modern AI applications."

**Trade-off:**
- Discrete: More searchable, more storage
- Summary: Less storage, preserves context

---

**Scenario 2: User Onboarding (Preferences Strategy)**

```
New student onboarding flow:
- Communication preferences
- Learning style preferences
- Schedule preferences
- Notification preferences
```

**Preferences Strategy:** Focuses on extracting preferences
- "User prefers email over SMS notifications"
- "User prefers morning study sessions"
- "User prefers video content over text"

**Why Preferences Strategy:**
- Optimized for preference extraction
- Builds user profile efficiently
- Personalization-focused

---


### How Strategies Work Behind the Scenes

**Discrete Strategy (Default):**
```
Conversation Messages
    ↓
[Background Worker]
    ↓
Extract individual facts using LLM
    ↓
Store each fact as separate long-term memory
    ↓
Vector index for semantic search
```

**Summary Strategy:**
```
Conversation Messages
    ↓
[Background Worker]
    ↓
Summarize conversation using LLM
    ↓
Store summary as long-term memory
    ↓
Vector index for semantic search
```

**📚 Learn More:** See the [Memory Extraction Strategies Guide](https://redis.github.io/agent-memory-server/memory-extraction-strategies/) for detailed examples and hands-on demos in Notebook 2.

---



### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Understand Session Management**
- Working memory persists within a session (like ChatGPT conversations)
- New sessions start with empty working memory
- Important facts should be in long-term memory for cross-session access
- Consider providing ways to resume or load previous session context

**5. Plan for Context Window Limits**
- Working memory doesn't expire, but can grow too large
- LLMs have context window limits (e.g., 128K tokens)
- Use compression strategies when conversations get long (covered in Notebook 03)
- Monitor token usage in long conversations

**6. Test Cross-Session Behavior**
- Verify long-term memories are accessible across sessions
- Test both same-session returns and new-session starts
- Ensure personalization works in both scenarios

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- Persists within the session (like ChatGPT conversations)
- Challenge: Can grow too large for context window limits

**Long-term Memory (Cross-Session):**
- Persistent knowledge: user preferences, domain facts, business rules
- Enables personalization AND consistent application behavior
- Can be user-scoped (personalization) or application-scoped (domain knowledge)
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **Session management** - Efficient storage and retrieval of conversation history

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

### **7. Memory Management Best Practices**

**Choose the Right Memory Type:**
- **Semantic** for facts and preferences (most common)
- **Episodic** for time-bound events and timeline
- **Message** for context-rich conversations (use sparingly)

**Understand Memory Lifecycle:**
- **Working memory:** Session-scoped, persists within session
- **Long-term memory:** Indefinite persistence, user-scoped, cross-session
- **Automatic extraction:** Trust the system to extract important facts
- **Context window limits:** Working memory can grow too large (use compression strategies)

**Benefits of Proper Memory Management:**
- ✅ **Natural conversations** - Users don't repeat themselves
- ✅ **Cross-session personalization** - Knowledge persists over time
- ✅ **Efficient storage** - Automatic deduplication prevents bloat
- ✅ **Semantic search** - Find relevant memories without exact keywords
- ✅ **Scalable** - Redis-backed, production-ready architecture

**Key Principle:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations.

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search_courses()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search_courses()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀

---

## 📚 Additional Resources

- [Agent Memory Server Documentation](https://github.com/redis/agent-memory-server) - Production-ready memory management
- [Agent Memory Client](https://pypi.org/project/agent-memory-client/) - Python client for Agent Memory Server
- [RedisVL Documentation](https://redisvl.com/) - Redis Vector Library
- [LangChain Guide](https://python.langchain.com/docs/modules/memory/) - Langchain
